In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split 

c:\Users\Ayush\apython\envs\gpu\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
# Load the dataset
df = pd.read_csv(r'C:\Users\Ayush\OneDrive\Desktop\AI-search-algo\ai-project\datasets\Crop_recommendation.csv')

# Drop any rows with missing values
df = df.dropna()

# Encode categorical variables if needed (e.g., using one-hot encoding)
# Example:
# df = pd.get_dummies(df, columns=['categorical_column'])

# Split the data into features (X) and labels (y)
X = df.drop(['label'], axis=1)  # Features excluding the 'label' column
y = df['label']  # Target variable 'label'

# Scale the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform labels in y_train and y_test
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert encoded labels to tensors
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.long).to(device)
y_test_tensor = torch.tensor(y_test_encoded, dtype=torch.long).to(device)

# Define the Q-learning model with increased complexity
class QLearningModel(nn.Module):
    def __init__(self, input_size, num_actions):
        super(QLearningModel, self).__init__()
        self.dense1 = nn.Linear(input_size, 128)
        self.relu1 = nn.ReLU()
        self.dense2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.dense3 = nn.Linear(64, num_actions)

    def forward(self, x):
        x = self.relu1(self.dense1(x))
        x = self.relu2(self.dense2(x))
        return self.dense3(x)

# Initialize the Q-learning model and move it to CUDA if available
model = QLearningModel(X_train.shape[1], len(label_encoder.classes_))
model.to(device)

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)

# Train the model
num_epochs = 200  # Increased number of epochs
training_loss_values = []  # List to store training loss values

for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    training_loss_values.append(loss.item())  # Append current loss to the list

# Save the trained model, scaler, and label encoder
torch.save(model.state_dict(), 'model.pth')
torch.save(scaler, 'scaler.pkl')
torch.save(label_encoder, 'label_encoder.pkl')

# Evaluate the model on the test set
with torch.no_grad():
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_pred = model(X_test_tensor)
    predicted_labels = torch.argmax(y_pred, dim=1)
    accuracy = (predicted_labels == y_test_tensor).sum().item() / len(y_test_tensor)
    print(f"Test Accuracy: {accuracy:.4f}")

# Create the line plot for training loss over epochs
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(1, num_epochs + 1)), y=training_loss_values, mode='lines', name='Training Loss'))
fig.update_layout(title='Training Loss Over Epochs', xaxis_title='Epoch', yaxis_title='Loss')
fig.show()


c:\Users\Ayush\apython\envs\gpu\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Test Accuracy: 0.9205


In [4]:
test_crop_labels_distribution = y_test.value_counts()

# Create the pie chart
fig = go.Figure()
fig.add_trace(go.Pie(labels=test_crop_labels_distribution.index, values=test_crop_labels_distribution.values))
fig.update_layout(title='Test Crop Labels Distribution')
fig.show()


In [5]:
feature1_name = 'N'  # Replace 'N' with the first feature name
feature2_name = 'P'  # Replace 'P' with the second feature name

# Create the scatter plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=X[feature1_name], y=X[feature2_name], mode='markers', name=f'{feature1_name} vs {feature2_name}'))
fig.update_layout(title=f'{feature1_name} vs {feature2_name}', xaxis_title=feature1_name, yaxis_title=feature2_name)
fig.show()


In [6]:
feature_name = 'N'  # Replace 'N' with the actual feature name you want to visualize

# Create the histogram
fig = go.Figure()
fig.add_trace(go.Histogram(x=X[feature_name], name=feature_name))
fig.update_layout(title=f'Histogram of {feature_name}', xaxis_title=feature_name, yaxis_title='Count')
fig.show()


In [7]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(1, num_epochs + 1)), y=training_loss_values, mode='lines', name='Training Loss'))
fig.update_layout(title='Training Loss Over Epochs', xaxis_title='Epoch', yaxis_title='Loss')
fig.show()



In [8]:
reward_values = [-loss for loss in training_loss_values]

# Create the line plot for training reward over epochs
fig = go.Figure() 
fig.add_trace(go.Scatter(x=list(range(1, num_epochs + 1)), y=reward_values, mode='lines', name='Training Reward'))
fig.update_layout(title='Training Reward Over Epochs', xaxis_title='Epoch', yaxis_title='Reward')
fig.show()

In [9]:
# Reverse the training loss values and epoch values

reversed_epochs = list(range(num_epochs, 0, -1))

# Create the line plot for reversed training loss over epochs
fig = go.Figure()
fig.add_trace(go.Scatter(x=reversed_epochs, y=training_loss_values, mode='lines', name='Training Loss'))
fig.update_layout(title='Reward Over Epochs', xaxis_title='Epoch', yaxis_title='Reward')
fig.show()


In [10]:
crop_labels_distribution = df['label'].value_counts()

# Create the bar chart
fig = go.Figure()
fig.add_trace(go.Bar(x=crop_labels_distribution.index, y=crop_labels_distribution.values, name='Crop Labels'))
fig.update_layout(title='Crop Labels Distribution', xaxis_title='Crop Label', yaxis_title='Count')
fig.show()


In [11]:
(X_train.shape[0])

1760

In [12]:
df['label'].unique()

array(['rice', 'maize', 'chickpea', 'kidneybeans', 'pigeonpeas',
       'mothbeans', 'mungbean', 'blackgram', 'lentil', 'pomegranate',
       'banana', 'mango', 'grapes', 'watermelon', 'muskmelon', 'apple',
       'orange', 'papaya', 'coconut', 'cotton', 'jute', 'coffee'],
      dtype=object)